# Generic Aperture Offsets Notebook

This notebook provides a general user interface for doing TA in some some target, and then placing some other target on the reference position of some other aperture.

In the future, the option to add multiple targets in the FOV, and to toggle between IDL coordinate output and Offset Command output will be implemented.

Warning: Confirmation plots do not currently work with this notebook.

In [1]:
# for development purposes
%load_ext autoreload
%autoreload 2

In [2]:
from all_jwst_offset_ta import utils

## Usage

1. Run the cell below, which will generate an interactive widget in the output space.
2. Select your instrument and science aperture from the drop-down menus.
   - The science aperture names can be a bit opaque, but you should be able to guess which one corresponds to your observation.
   - If you have access to data that uses the subarray of interest, you can check the header for the "APERNAME" there and match it.
3. Enter the RA and Dec coordinates of your acquisition and science targets.
   - the frame must be 'ICRS'
   - the units must be in decimal degrees
4. Enter the PA angle of the V3 axis of the telescope at the instrument reference point.
   - In APT, this corresponds to the "Specify PA range using V3PA" option under Special Requirements.
   - This angle is given in data headers as "ROLL_REF"
5. If you wish to know the positions of other stars in the field of view, you can enter them in the "Other stars" box
   - The format is `label: (ra, dec)` where `ra` and `dec` are given in decimal degrees
   - There is currently no error checking for formatting; you will just make it crash/get a wrong answer
   - For multi-line entries, you may need to type them elsewhere and they copy-paste the list into the box
7. Press "Compute offset" to get the offsets to enter into APT (you may need to press it twice).
   - You will also get the positions in IDL coordinates of all the stars before and after the slew.
   - For same-target TA, a hack is to enter the same position for the ACQ and SCI target

Copy the lines below and paste them into the `Other stars` box, as an example:
```
a: (1.0, 2.0)
b: (2.0, 3.0)
c: (3.0, 4.0)
```

## Initialization with default values

In [3]:
# create the ComputeOffsets object and start the UI
co1 = utils.ComputeOffsets()
ui1 = co1.show_ui()
ui1

## Initialization from a configuration dictionary

Alternatively, one can provide a dictionary to pre-populate the fields of the interface. This is useful for repeatability/record keeping purposes.

As an example, let's use MIRI's 15.5 um coronagraph to do TA on a star at (90, 90) and then slew to (91, 89) at a V3_PA angle of 180 deg. There are three other stars of interest, labeled a, b, and c, located at coordinates (1, 2), (2, 3), and (3, 4).

In [9]:
initial_values={
    'instr': 'miri',
    'sci_aper': 'mirim_mask1550', 
    'pa': 180.,
    'acq_ra': 90., 'acq_dec': 90.,
    'sci_ra': 91., 'sci_dec': 89.,
}
# add a multi-line string of the other stars
initial_values['other_stars'] = """
a: (1.0, 2.0)
b: (2.0, 3.0)
c: (3.0, 4.0)
""".strip()

In [10]:
co2 = utils.ComputeOffsets(initial_values=initial_values)
ui2 = co2.show_ui()
ui2

A caveat: for each set of calculations, the user should create a new "ComputeOffsets" object. Calling the same UI object again changes its values globally.